# Pylablibsm API Example:

This file provides an example of using the calibration automation state-machine API

Sam Condon \
Caltech \
06/14/2021

In [1]:
import sys
import numpy as np
from inspect import signature
sys.path.append(r'..\\pylab\\')
from statemachine import State, StateMachine

from pylablib.pylablibsm import SM

## Actions

The state machine on its own does not execute any useful code. It simply provides a framework through which 
instrument control code can plug in through what is termed an 'action.' **An action is a collection of two pieces of data:**

*1) Function pointer* \
*2) Identifier string*

When actions are added to the state-machine, we pass both of the above pieces of information to the **\<state_machine_object\>.add_action_to_state()** method. Actions must be assigned to an individual state-machine state, so we also pass in a string to identify which state the action should execute within. Overall the call signature for the add_action() method is as follows:

*\<state_machine_object\>.add_action_to_state(\<state identifier string\>, \<action identifier string\>, \<action function pointer\>)*

All actions must accept a single positional argument. In the example actions below, this argument has been called 'action_dict.' When an action is called within the state-machine, it is passed a dictionary through the action_dict argument. This dictionary takes the following form:

action_dict = {'state_machine': \<state machine instance object\>, 'params': \<action parameters\>}

The 'state_machine' key gives access to the state-machine instance object allowing actions to trigger state transitions. The 'params' key gives access to any parameters previously passed in to the action.


In [2]:
def init_action(action_dict):
    print("This is the function that initializes communication interfaces with all instruments.")

def waiting_action(action_dict):    
    print("Here we are waiting for user input.")
    print("*Query instruments for lost connections*")

def thinking_action(action_dict):
    print("This is the thinking action that generates the control loop parameters based on user input")
    
    params = action_dict['params']
    
    waves = np.arange(params['start_wavelength'], params['stop_wavelength'], 
                      params['step_size'])
    
    return waves


class ControlLoopActions:
    
    def __init__(self):
        self.waves = None
        self.waves_ind = 0
        self.control_loop_complete = False       
    
    def set_waves(self, waves):
        self.waves = waves
    
    def moving_action(self, action_dict):
        print("Moving monochromator to wavelength: {} um.".format(self.waves[self.waves_ind]))
        self.waves_ind += 1
        if not self.waves_ind < len(self.waves):
            self.control_loop_complete = True
        action_dict['state_machine'].control_loop_next()
    
    def measuring_action(self, action_dict):
        print("*Take an exposure*")
        action_dict['state_machine'].control_loop_next()

    def checking_action(self, action_dict):
        print("*Check exposure pixel data*")
        action_dict['state_machine'].control_loop_next()

    def compressing_action(self, action_dict):
        print("*Compress pixel data*")
        action_dict['state_machine'].control_loop_next()

    def writing_action(self, action_dict):
        print("*Write compressed data to server*")
        action_dict['state_machine'].control_loop_next()

    def resetting_action(self, action_dict):
        print("*Reset detector, close monochromator shutter*")
        print("\n")
        if self.control_loop_complete == True:
            print("Control loop complete!")
            action_dict['state_machine'].control_loop_complete()
        else:
            action_dict['state_machine'].control_loop_next()
         


#### Add all actions to the state machine using the call signature indicated in the comments above:

In [3]:
#instantiate state machine
state_machine = SM()

control_loop = ControlLoopActions()

# Add all desired actions to the state-machine###########################
state_machine.add_action_to_state('Initializing', 'init_action_0', init_action)
state_machine.add_action_to_state('Waiting', 'waiting_action_0', waiting_action)
state_machine.add_action_to_state('Thinking', 'thinking_action_0', thinking_action)
state_machine.add_action_to_state('Moving', 'moving_action_0', control_loop.moving_action)
state_machine.add_action_to_state('Measuring', 'measuring_action_0', control_loop.measuring_action)
state_machine.add_action_to_state('Checking', 'checking_action_0', control_loop.checking_action)
state_machine.add_action_to_state('Compressing', 'compressing_action_0', control_loop.compressing_action)
state_machine.add_action_to_state('Writing', 'writing_action_0', control_loop.writing_action)
state_machine.add_action_to_state('Resetting', 'resetting_action_0', control_loop.resetting_action)



#### Start the state-machine by sending it into the Initializing state:
Note that any actions added above to the Initializing state will be executed after the state machine is started

In [4]:
state_machine.start_machine()

Initializing
This is the function that initializes communication interfaces with all instruments.


#### After Initializing, enter the waiting state:

In [5]:
state_machine.init_to_wait()

Waiting


True

Here we are waiting for user input.
*Query instruments for lost connections*


#### Now assume that the waves_dict dictionary are parameters input from a user. Pass these to the thinking action, then move from the waiting to thinking state:

In [6]:
waves_dict = {'start_wavelength': 0.7, 'stop_wavelength': 1.5, 'step_size': 0.1}
state_machine.set_action_parameters('Thinking', 'thinking_action_0', waves_dict)

state_machine.wait_to_think()

Thinking


True

This is the thinking action that generates the control loop parameters based on user input


#### The thinking action that we defined generates a numpy array with wavelength values used to specify the behavior of the control loop. Get this returned data and pass it to the control loop:

In [7]:
waves = state_machine.get_state_action_data('Thinking')['thinking_action_0']
control_loop.set_waves(waves)

#### Now that the parameters of the control loop have been set, start the control loop:

In [8]:
state_machine.start_control_loop()

Moving


True

Moving monochromator to wavelength: 0.7 um.
Measuring
*Take an exposure*
Checking
*Check exposure pixel data*
Compressing
*Compress pixel data*
Writing
*Write compressed data to server*
Resetting
*Reset detector, close monochromator shutter*


Moving
Moving monochromator to wavelength: 0.7999999999999999 um.
Measuring
*Take an exposure*
Checking
*Check exposure pixel data*
Compressing
*Compress pixel data*
Writing
*Write compressed data to server*
Resetting
*Reset detector, close monochromator shutter*


Moving
Moving monochromator to wavelength: 0.8999999999999999 um.
Measuring
*Take an exposure*
Checking
*Check exposure pixel data*
Compressing
*Compress pixel data*
Writing
*Write compressed data to server*
Resetting
*Reset detector, close monochromator shutter*


Moving
Moving monochromator to wavelength: 0.9999999999999999 um.
Measuring
*Take an exposure*
Checking
*Check exposure pixel data*
Compressing
*Compress pixel data*
Writing
*Write compressed data to server*
Resetting
*Reset